<div style="font-size:40px">
    ****Classification****
    </div>
    <br>

This notebook helps me get start, how to preproccess the data,
how to apply different algorithm, and test their accuracy, choose the best one.
(Still undergo)

Excuses for English mistakes.


<div style="font-size:30px">
**1. Data Preprocessing**
</div>
<br>
We start by preprocessing the data and visualize it.

First we import libraries and read it.

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Load Data
data = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
data.head()

In [ ]:
sns.pairplot(data, hue='Outcome')

In [ ]:
#First lets split the data
X = data.iloc[:, :-1].values
y = data.iloc[:, 8].values
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

<div style="font-size:25px">
**1.1 Handling Nulls**
</div>
<br>

Datasets can contain null or(zeros)  values, whatever is the kind of the problem, no model can handle this NULL.

So we start by checking which columns contains null values

In [ ]:
#Checking for nulls
data.isnull()
data.isnull().sum()
data.eq(0).any().any()

It seems there is no null data in this dataset. But there is zeros

In such a case where there is nulls, the easiest way to solve this problem, is by dropping the rows or columns that contain null values (dataset.dropna())

However it is not the best option, it can lead to loss of valuable information.
So we can use **Imputation**
This is simply the process of substituting the missing values of our dataset.

In [ ]:
#Imputation
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values=0, strategy='mean', axis=0)
X_train = imputer.fit_transform(X_train)
X_test = imputer.fit_transform(X_test)

<div style="font-size:25px">
    **1.2 Standarization**
 </div>
    <br>
We transform our values such that the **mean** of the values is 0 and **standard deviation** is 1.

Remeber the table above the head of data, we can see that for example, we have 2 numerical values **Pregnancies** and **Glucose**. They are not on the same scale and it will always be that glucose is greater than pregnancies, therefore the model will give more weightage to glucose which is not the ideal case as pregnancies is also a factor here. So to avoid that we perform standarization

In [ ]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
X_train = std.fit_transform(X_train)
X_test = std.transform(X_test)

<div style="font-size:25px">
    **1.3 Handling Categorical Variables**
 </div>
    <br>
    
Categorical variables are that contain discrete values, like color of an item. They are divided into 2 types
* **Ordnial categorical variables**  these are variables that can be orderd like size of a T-shirt 
* **Nominal categorical variables**  these are variables can't be orderd like color

So here we need to prerpocess them differently

For Ordinal we use Label Encoder from sklearn
For nominal we use One-Hot Encoding

We will not be doing it on this dataset

<div style="font-size:25px">
    **1.4. Multicollinearity and its impact**
 </div>
    <br>
Multicollinearity occurs in our dataset when we have features which are strongly dependent on each other.    
* The main impact it will have is that it can cause the decision boundary to change which can have a huge impact on the result of our model.
* In addition to that if we have multicollinearity in our dataset then we won’t be able to use our weight vector to calculate the feature importance.

In [ ]:
def plot_corr(data, size):
    corr = data.corr()
    fig, ax = plt.subplots(figsize=(size,size))
    ax.matshow(corr)
    plt.xticks(range(len(corr.columns)), corr.columns)
    plt.yticks(range(len(corr.columns)), corr.columns)

plot_corr(data, data.shape[1])
data.corr()

In [ ]:
sns.heatmap(data.corr(), annot=True)

If we look at the previous outputs, we can easily identify if there is unrelated patterns between columns.
In our case the data seem related

<div style="font-size:30px">
    **2. Learning**
 </div>
    <br>
1. Logistic regression
2. KNN
3. Decision Tree
4. Naive Bayes
5. Neural Network

<div style="font-size:20px">
    **2.1 Logistic Regression**
</div>
* One of the limitations of Logistic Regression is the fact that it can only categorize data with two distinct classes. 
* When we train our model, we are in fact attempting to select the Sigmoid function whose shape best fits our data.

In [ ]:
#Fitting model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

In [ ]:
#Predicting test result
y_pred = classifier.predict(X_test)

#Predicting training result
y_pred_train = classifier.predict(X_train)

In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
true_pred = cm[1][1] + cm[0][0]
neg_pred = cm[1][0] + cm[0][1]
print(cm)
print('True prediction: ', true_pred, ' False prediction: ', neg_pred)
print('Accuracy of true prediction: ', (true_pred/X_test.shape[0])*100, '%')
print('Accuracy of false prediction: ', (neg_pred/X_test.shape[0])*100, '%')

In [ ]:
#Performance on training data
from sklearn import metrics
print ("Accuracy: ({0:.4f})".format(metrics.accuracy_score(y_train, y_pred_train)))

#Performance on testing data
from sklearn import metrics
print ("Accuracy: ({0:.4f})".format(metrics.accuracy_score(y_test, y_pred)))

<div style="font-size:20px">
    **2.2 KNN**
</div>
* KNN is a non-parametric, lazy learning algorithm. 
* Needs high memory requirement
* Sensitive to irrelevant features

In [ ]:
#Fitting model
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(X_train, y_train)

In [ ]:
#Predicting test result
y_pred_knn = knn.predict(X_test)

#Predicting training result
y_pred_train_knn = knn.predict(X_train)

In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_knn)
true_pred = cm[1][1] + cm[0][0]
neg_pred = cm[1][0] + cm[0][1]
print(cm)
print('True prediction: ', true_pred, ' False prediction: ', neg_pred)
print('Accuracy of true prediction: ', (true_pred/X_test.shape[0])*100, '%')
print('Accuracy of false prediction: ', (neg_pred/X_test.shape[0])*100, '%')

In [ ]:
#Performance on training data
from sklearn import metrics
print ("Accuracy: ({0:.4f})".format(metrics.accuracy_score(y_train, y_pred_train_knn)))

#Performance on testing data
from sklearn import metrics
print ("Accuracy: ({0:.4f})".format(metrics.accuracy_score(y_test, y_pred_knn)))

<div style="font-size:20px">
    **2.3 Decision Tree**
</div>
* In order to determine which of the three splits is better, we use impurity.
* We ultimately decide on the split with the largest information gain.

In [ ]:
#Fitting model
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

In [ ]:
#Predicting test result
y_pred_tree = dt.predict(X_test)

#Predicting training result
y_pred_train_tree = dt.predict(X_train)

In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_tree)
true_pred = cm[1][1] + cm[0][0]
neg_pred = cm[1][0] + cm[0][1]
print(cm)
print('True prediction: ', true_pred, ' False prediction: ', neg_pred)
print('Accuracy of true prediction: ', (true_pred/X_test.shape[0])*100, '%')
print('Accuracy of false prediction: ', (neg_pred/X_test.shape[0])*100, '%')

In [ ]:
#Display Tree
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO 
from IPython.display import Image 
from pydot import graph_from_dot_data
feature_names = data.columns[:-1]
dot_data = StringIO()
export_graphviz(dt, out_file=dot_data, feature_names=feature_names)
(graph, ) = graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

<div style="font-size:20px">
    **2.4 Naive Bayes**
</div>
* Naive Bayes Classifier is based on the Bayes’ Theorem of conditional probability

In [ ]:
#Fitting model
from sklearn.naive_bayes import GaussianNB   
NB = GaussianNB()  
NB.fit(X_train, y_train)   

In [ ]:
#Predicting test result
y_pred_nb = NB.predict(X_test)

#Predicting training result
y_pred_train_nb = NB.predict(X_train)

In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_nb)
true_pred = cm[1][1] + cm[0][0]
neg_pred = cm[1][0] + cm[0][1]
print(cm)
print('True prediction: ', true_pred, ' False prediction: ', neg_pred)
print('Accuracy of true prediction: ', (true_pred/X_test.shape[0])*100, '%')
print('Accuracy of false prediction: ', (neg_pred/X_test.shape[0])*100, '%')

In [ ]:
#Performance on training data
from sklearn import metrics
print ("Accuracy: ({0:.4f})".format(metrics.accuracy_score(y_train, y_pred_train_nb)))

#Performance on testing data
from sklearn import metrics
print ("Accuracy: ({0:.4f})".format(metrics.accuracy_score(y_test, y_pred_nb)))

<div style="font-size:20px">
    **2.5 Neural Network**
</div>
* We will use keras to build our neural network.
* There are two main types of models available in keras — Sequential and Model. we will use Sequential model to build our neural network.
* We use Dense library to build input, hidden and output layers of a neural network.
* We have 8 input features and one target variable. 2 Hidden layers. Each hidden layer will have 4 nodes.
* ReLu will be the activation function for hidden layers. As this is a binary classification problem we will use sigmoid as the activation function.

In [ ]:
from keras import Sequential
from keras.layers import Dense

classifierNN = Sequential()

#output = activation(dot(input, kernel) + bias)
#kernel is the weight matrix. kernel initialization defines the way to set the initial random weights of Keras layers.
#Random normal initializer generates tensors with a normal distribution.

#First Hidden Layer
classifierNN.add(Dense(8, activation='relu', kernel_initializer='random_normal', input_dim=8))

#Second Hidden Layer
classifierNN.add(Dense(8, activation='relu', kernel_initializer='random_normal'))

#Output Layer
classifierNN.add(Dense(1, activation='sigmoid',  kernel_initializer='random_normal'))

#Compiling the neural network
#We use binary_crossentropy to calculate the loss function between the actual output and the predicted output.
#Adam stands for Adaptive moment estimation. Adam is a combination of RMSProp + Momentum.
#Momentum takes the past gradients into account in order to smooth out the gradient descent.
#We use accuracy as the metrics to measure the performance of the model
classifierNN.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])


#Fitting the data to the training dataset
#We use a batch_size of 10. This implies that we use 10 samples per gradient update.3
#We iterate over 100 epochs to train the model. An epoch is an iteration over the entire data set.
classifierNN.fit(X_train,y_train, batch_size=10, epochs=100)

In [ ]:
eval_model=classifierNN.evaluate(X_train, y_train)
eval_model

In [ ]:
y_pred_NN=classifierNN.predict(X_test)
y_pred_NN =(y_pred_NN>0.5)

In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_NN)
true_pred = cm[1][1] + cm[0][0]
neg_pred = cm[1][0] + cm[0][1]
print(cm)
print('True prediction: ', true_pred, ' False prediction: ', neg_pred)
print('Accuracy of true prediction: ', (true_pred/X_test.shape[0])*100, '%')
print('Accuracy of false prediction: ', (neg_pred/X_test.shape[0])*100, '%')

<div style="font-size:25px">
    ****Refrences****
</div>
<br>
* Data preprocessing: https://towardsdatascience.com/introduction-to-data-preprocessing-in-machine-learning-a9fa83a5dc9d
* Logistic regression: https://towardsdatascience.com/logistic-regression-python-7c451928efee
* Testing model accuracy: https://medium.com/@sachinkmr375/testing-the-models-accuracy-in-ml-8385ee944e3f
* Data preprocessing: https://medium.com/@sachinkmr375/selecting-an-algorithm-for-processing-training-data-e6114f91f6c5
* KNN: https://towardsdatascience.com/k-nearest-neighbor-python-2fccc47d2a55
* Decision Tree: https://towardsdatascience.com/decision-tree-in-python-b433ae57fb93
* Naive Bayes: https://medium.com/dataseries/lets-build-your-first-naive-bayes-classifier-with-python-d31a5140e4bc
* Neural Network: https://medium.com/datadriveninvestor/building-neural-network-using-keras-for-classification-3a3656c726c1